In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from dotenv import load_dotenv
import warnings

# === Load Environment & Config ===
load_dotenv()
# Zid Settings
ZID_HMBR = int(os.environ["ZID_GULSHAN_TRADING"])
ZID_GI = int(os.environ["ZID_GI"])
ZID_ZEPTO = int(os.environ["ZID_ZEPTO_CHEMICALS"])  # Zepto business
ZID_HMBR_ONLINE = int(os.environ["ZID_HMBR_ONLINE_SHOP"])  # HMBR Online business
ZID_PACKAGING = int(os.environ["ZID_GULSHAN_PACKAGING"])  # Packaging business

# Project settings
PROJECT_TRADING = os.environ["PROJECT_100001"]
PROJECT_GI = os.environ["PROJECT_100000"]   
PROJECT_ZEPTO = os.environ["PROJECT_100005"]
PROJECT_HMBR_ONLINE = os.environ["PROJECT_100007"]
PROJECT_PACKAGING = os.environ["PROJECT_100009"]


PROJECT_ROOT = os.path.dirname(os.getcwd())
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from mail import send_mail, get_email_recipients
from project_config import DATABASE_URL

engine = create_engine(DATABASE_URL)
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
pd.set_option('display.float_format', '{:.2f}'.format)


def get_sales_COGS(zid,start_date,end_date):
    df = pd.read_sql("""SELECT caitem.zid,caitem.xitem,caitem.xdesc,caitem.xgitem, (imtrn.xqty*imtrn.xsign) as qty, (imtrn.xval*imtrn.xsign) as totalvalue,
                        opddt.xqty as opddt_qty,opddt.xrate,opddt.xlineamt,(opddt.xdtwotax-opddt.xdtdisc) as xdtwotax  
                        FROM caitem
                        JOIN imtrn
                        ON caitem.xitem = imtrn.xitem
                        JOIN opddt
                        ON (imtrn.xdocnum = opddt.xdornum AND imtrn.xitem = opddt.xitem AND imtrn.xdocrow = opddt.xrow)
                        JOIN opdor
                        ON imtrn.xdocnum = opdor.xdornum
                        WHERE caitem.zid = '%s'
                        AND imtrn.zid = '%s'
                        AND opddt.zid = '%s'
                        AND opdor.zid = '%s'
                        AND imtrn.xdoctype = '%s'
                        AND imtrn.xdate >= '%s'
                        AND imtrn.xdate <= '%s'"""%(zid,zid,zid,zid,'DO--',start_date,end_date),con=engine)
    return df

def get_return(zid,start_date,end_date):
    df = pd.read_sql("""SELECT imtrn.xitem, imtrn.xqty, (imtrn.xval*imtrn.xsign) as returnvalue, opcdt.xrate, (opcdt.xrate*imtrn.xqty) as totamt
                        FROM imtrn 
                        JOIN opcdt
                        ON imtrn.xdocnum = opcdt.xcrnnum
                        AND imtrn.xitem = opcdt.xitem
                        JOIN opcrn
                        ON imtrn.xdocnum = opcrn.xcrnnum
                        WHERE imtrn.zid = '%s'
                        AND opcdt.zid = '%s'
                        AND opcrn.zid = '%s'
                        AND imtrn.xdoctype = '%s'
                        AND imtrn.xdate >= '%s'
                        AND imtrn.xdate <= '%s'"""%(zid,zid,zid,'SR--',start_date,end_date),con=engine)
    return df

def get_gl_details(zid,COGS,start_date,end_date):
    df = pd.read_sql("""SELECT glmst.xacctype, SUM(gldetail.xprime)
                        FROM glmst
                        JOIN gldetail
                        ON glmst.xacc = gldetail.xacc
                        JOIN glheader
                        ON gldetail.xvoucher = glheader.xvoucher
                        WHERE glmst.zid = '%s'
                        AND gldetail.zid = '%s'
                        AND glheader.zid = '%s'
                        AND (glmst.xacctype = 'Income' OR glmst.xacctype = 'Expenditure')
                        AND glmst.xacc != '%s'
                        AND glheader.xdate >= '%s'
                        AND glheader.xdate <= '%s'
                        GROUP BY glmst.xacctype"""%(zid,zid,zid,COGS,start_date,end_date),con = engine)
    return df

def get_gl_details_zepto(zid,COGS,MRP,start_date,end_date):
    df = pd.read_sql("""SELECT glmst.xacctype, SUM(gldetail.xprime)
                        FROM glmst
                        JOIN gldetail
                        ON glmst.xacc = gldetail.xacc
                        JOIN glheader
                        ON gldetail.xvoucher = glheader.xvoucher
                        WHERE glmst.zid = '%s'
                        AND gldetail.zid = '%s'
                        AND glheader.zid = '%s'
                        AND (glmst.xacctype = 'Income' OR glmst.xacctype = 'Expenditure')
                        AND glmst.xacc != '%s'
                        AND glmst.xacc != '%s'
                        AND glheader.xdate >= '%s'
                        AND glheader.xdate <= '%s'
                        GROUP BY glmst.xacctype"""%(zid,zid,zid,COGS,MRP,start_date,end_date),con = engine)
    return df

def get_gl_details_project(zid,project,start_date,end_date,COGS):
    df = pd.read_sql("""SELECT glmst.xacctype, SUM(gldetail.xprime)
                        FROM glmst
                        JOIN gldetail
                        ON glmst.xacc = gldetail.xacc
                        JOIN glheader
                        ON gldetail.xvoucher = glheader.xvoucher
                        WHERE glmst.zid = '%s'
                        AND gldetail.zid = '%s'
                        AND glheader.zid = '%s'
                        AND gldetail.xproj = '%s'
                        AND (glmst.xacctype = 'Income' OR glmst.xacctype = 'Expenditure')
                        AND glmst.xacc != '%s'
                        AND glheader.xdate >= '%s'
                        AND glheader.xdate <= '%s'
                        GROUP BY  glmst.xacctype"""%(zid,zid,zid,project,COGS,start_date,end_date),con = engine)
    return df


COGS_zepto = '04010020'

COGS = '04010020'

end_date = (datetime.now() - timedelta(days = 2)).strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days = 33)).strftime('%Y-%m-%d')

print (start_date, end_date)

🔧 ENV=production: Using .env (localhost)
2025-08-11 2025-09-11


In [6]:

############ZEPTO ################
######    xlineamt to xdtwotax, COGS should be COGS_zepto
df_sales_5 = get_sales_COGS(ZID_ZEPTO,start_date,end_date)
df_sales_5 = df_sales_5.groupby(['xitem','xdesc'])['totalvalue','xdtwotax'].sum().reset_index().round(1)
df_return_5 = get_return(ZID_ZEPTO,start_date,end_date)
df_return_5 = df_return_5.groupby(['xitem'])['returnvalue','totamt'].sum().reset_index().round(1)
df_final_5 = df_sales_5.merge(df_return_5[['xitem','returnvalue','totamt']],on=['xitem'],how='left').fillna(0)
df_final_5['final_sales'] = df_final_5['xdtwotax'] - df_final_5['totamt']
df_final_5['final_cost'] = df_final_5['totalvalue'] + df_final_5['returnvalue']
df_final_5 = df_final_5.drop(['xdtwotax','totamt'],axis=1)
df_final_5 = df_final_5.drop(['returnvalue','totalvalue'],axis=1)
df_final_5['Gross_Profit'] = df_final_5['final_sales'] + df_final_5['final_cost']
df_final_5['Profit_Ratio'] = np.where(df_final_5['final_sales'] != 0, (df_final_5['Gross_Profit'] / df_final_5['final_sales']) * 100, np.nan)
df_final_5 = df_final_5.sort_values(by=['Profit_Ratio']).reset_index(drop=True) ########added
df_final_5.loc[len(df_final_5.index),:]=df_final_5.sum(axis=0,numeric_only = True)
df_final_5.at[len(df_final_5.index)-1,'xdesc'] = 'Total_Item_Profit'
df_pl_5 = get_gl_details_zepto(ZID_ZEPTO,COGS_zepto,'07080001',start_date,end_date)
summary_5 = df_final_5.tail(1).drop('xitem',axis=1)
summary_5 = summary_5.to_dict('records')
df_pl_5 = df_pl_5.to_dict('records')
summary_5[0]['Income_gl'] = df_pl_5[0]['sum']
try:
    summary_5[0]['Expenditure_gl'] = df_pl_5[1]['sum']
except:
    summary_5[0]['Expenditure_gl'] = 0
# summary_5[0]['Expenditure_gl'] = df_pl_5[1]['sum']
main_data_dict[ZID_ZEPTO] = summary_5[0]
main_data_dict[ZID_ZEPTO]['Net']=main_data_dict[ZID_ZEPTO]['Gross_Profit']-main_data_dict[ZID_ZEPTO]['Expenditure_gl'] ######added
df_pl_5 = pd.DataFrame(df_pl_5)
df_final_5 = pd.concat([df_final_5, df_pl_5],axis=1)
main_data_dict["Zepto"] = main_data_dict.pop(100005)  ####### added
df_sales_5

C:\Users\Administrator\AppData\Local\Temp\11\ipykernel_6976\3763709493.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_sales_5 = df_sales_5.groupby(['xitem','xdesc'])['totalvalue','xdtwotax'].sum().reset_index().round(1)
C:\Users\Administrator\AppData\Local\Temp\11\ipykernel_6976\3763709493.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_return_5 = df_return_5.groupby(['xitem'])['returnvalue','totamt'].sum().reset_index().round(1)


NameError: name 'np' is not defined